# Sprint アンサンブル学習

# 1.このSprintについて

## Sprintの目的  
・アンサンブル学習について理解する

## どのように学ぶか  
・スクラッチでアンサンブル学習の各種手法を実装していきます。

# 2.アンサンブル学習  
3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。  
・ブレンディング  
・バギング  
・スタッキング  

### 小さなデータセットの用意
以前も利用した回帰のデータセットを用意します。  
House Prices: Advanced Regression Techniques  
この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。  
train.csvを学習用（train）8割、検証用（val）2割に分割してください。

### scikit-learn

単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。

sklearn.linear_model.LinearRegression — scikit-learn 0.21.3 documentation  
sklearn.svm.SVR — scikit-learn 0.21.3 documentation  
sklearn.tree.DecisionTreeRegressor — scikit-learn 0.21.3 documentation

# 3.ブレンディング

## 【問題1】ブレンディングのスクラッチ実装  
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

### ブレンディングとは

ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。  
最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。

手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）  
ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）  
入力データの前処理の仕方（例：標準化、対数変換、PCAなど）  

重要なのはそれぞれのモデルが大きく異なることです。  

回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。  

《補足》  
分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./HousePrice_train.csv')
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [3]:
drop_df = df
# 半分以上欠損値を含む列を削除する。
drop_df = drop_df.dropna(thresh=df.shape[0]*0.5, axis=1)

# 欠損値があるサンプル（行）は削除する。
drop_df = drop_df.dropna(axis=0)

In [4]:
df_data = drop_df.loc[:,["GrLivArea","YearBuilt","SalePrice"]]
df_data

,GrLivArea,YearBuilt,SalePrice
1,1262,1976,181500
2,1786,2001,223500
3,1717,1915,140000
4,2198,2000,250000
6,1694,2004,307000
...,...,...,...
1447,2090,1995,240000
1451,1578,2008,287090
1455,1647,1999,175000
1456,2073,1978,210000


In [5]:
import numpy as np
# ndarrayへ変換
X = np.array(df_data.iloc[:,0:2])
y = np.array(df_data.iloc[:,2])

In [29]:
# 訓練データと検証データの分割
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 47)

#### train_test_splitを実行するたびに、X_train, X_test, y_train, y_testが変動するためMSEスコアが変動するため保存する。

In [174]:
import pickle

with open("X_train.pkl", "wb") as f:
    pickle.dump(X_train, f)

with open("X_test.pkl", "wb") as f:
    pickle.dump(X_test, f)
    
with open("y_train.pkl", "wb") as f:
    pickle.dump(y_train, f)
    
with open("y_test.pkl", "wb") as f:
    pickle.dump(y_test, f)

In [30]:
# 標準化
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(X_train)
ss_X_train = ss.transform(X_train)
ss_X_test = ss.transform(X_test)

In [31]:
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error

In [32]:
#分類器をセット。ここではロジスティック回帰、ランダムフォレスト分類器、ガウシアンナイーブベースを用いる。
clf1 = LogisticRegression(multi_class='multinomial', random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()
clf4 = SVC(probability=True, random_state=1)

### 単一モデルで検証

#### ＜ロジスティック回帰＞

In [33]:
# 学習
ss_clf = clf1.fit(ss_X_train, y_train)

# 推定
y_pred_LR = clf1.predict(ss_X_test)

# 評価
err_LR = mean_squared_error(y_test, y_pred_LR)
err_LR


5321997491.7260275

#### ＜ランダムフォレスト＞

In [34]:
# 学習
ss_clf = clf2.fit(ss_X_train, y_train)

# 推定
y_pred_LF = clf2.predict(ss_X_test)

# 評価
err_LR = mean_squared_error(y_test, y_pred_LF)
err_LR


4414022744.479452

#### ＜ガウシアンナイーブベース＞

In [35]:
# 学習
ss_clf = clf3.fit(ss_X_train, y_train)

# 推定
y_pred_GNB = clf3.predict(ss_X_test)

# 評価
err_GNB = mean_squared_error(y_test, y_pred_GNB)
err_GNB


4824000759.534246

#### ＜SVC＞

In [36]:
# 学習
clf = clf4.fit(ss_X_train, y_train)

# 推定
y_pred_SVM = clf4.predict(ss_X_test)

# 評価
err_svc = mean_squared_error(y_test, y_pred_SVM)
err_svc


5800495447.20548

### アンサンブルモデルで検証

### <voting='hard'>

In [37]:
#アンサンブル学習器を作成。voting='hard'に設定し、単純な多数決で値を決めることにする。
eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
eclf1 = eclf1.fit(ss_X_train, y_train)


In [38]:
y_pred1 = eclf1.predict(ss_X_test)
y_pred1

array([170000, 270000, 118000, 208900, 160200, 174000, 230000, 250000,
       225000, 236000, 176000, 170000, 170000, 230000, 183500, 250000,
       250000, 250000, 219500, 275000, 170000, 174000, 133900, 140000,
       174000,  91500, 140000, 290000, 140000, 119000, 142000, 159000,
       240000, 174000, 140000, 170000, 250000, 220000, 143000, 250000,
       184750, 230000, 125000, 174000, 123000, 191000, 135000, 174000,
       119000, 144500, 140000, 140000, 174000, 176000, 174000, 214000,
       250000, 240000, 120000, 135000,  62383, 250000, 134000, 250000,
       176000, 230000, 151000, 174000, 174000, 290000, 178000, 192140,
       174000, 230000, 174000, 174000, 140000, 174000, 134000, 250000,
       250000, 250000, 240000, 170000, 183500, 176000, 230000, 230000,
       189000, 120000, 120000, 250000, 174000, 295000, 143000, 250000,
       250000, 170000, 225000, 190000, 230000, 140000, 170000, 170000,
       215000, 190000, 170000, 176000, 176000, 140000, 222500, 149500,
      

In [39]:
np.array_equal(eclf1.named_estimators_.lr.predict(X),
               eclf1.named_estimators_['lr'].predict(X))

True

In [40]:
err1 = mean_squared_error(y_test, y_pred1)
err1

5677414021.102739

### <voting='soft'>

In [41]:
eclf2 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft')
eclf2 = eclf2.fit(ss_X_train, y_train)

In [42]:
y_pred2 = eclf2.predict(ss_X_test)
y_pred2

array([196500, 270000, 120000, 208900, 192000, 244600, 306000, 271900,
       225000, 240000, 197000, 280000, 167500, 232000, 183500, 325300,
       250000, 340000, 219500, 275000, 181000, 279500, 185000, 157500,
       180000, 113000, 161000, 446261, 174500, 132000, 143000, 159000,
       265979, 207000, 177000, 239000, 380000, 311872, 179900, 225000,
       320000, 219210, 125000, 174000, 143000, 239000, 155000, 192000,
       119000, 144500, 144000, 180500, 192000, 208900, 181000, 214000,
       250000, 240000, 120000, 257500,  62383, 250000, 160000, 275500,
       230000, 392500, 181000, 192000, 245000, 290000, 178000, 275000,
       208900, 281000, 181000, 286000, 165000, 174000, 134000, 250000,
       269790, 285000, 266500, 172500, 183500, 187750, 281000, 237000,
       210000, 120000, 143000, 336000, 192000, 295000, 143000, 501837,
       250000, 170000, 225000, 190000, 231500, 146500, 176000, 170000,
       241500, 190000, 175000, 187750, 230000, 160000, 222500, 176000,
      

In [43]:
err2 = mean_squared_error(y_test, y_pred2)
err2

3741653515.712329

In [59]:
eclf21 = VotingClassifier(estimators=[('rf', clf2), ('gnb', clf3), ('svc', clf4)], voting='soft')
eclf21 = eclf21.fit(ss_X_train, y_train)

In [60]:
y_pred21 = eclf21.predict(ss_X_test)
y_pred21

array([196500, 270000, 120000, 208900, 192000, 244600, 306000, 271900,
       225000, 236000, 197000, 280000, 167500, 232000, 183500, 325300,
       250000, 340000, 219500, 275000, 181000, 279500, 185000, 157500,
       180000, 113000, 161000, 446261, 174500, 132000, 143000, 159000,
       265979, 207000, 177000, 239000, 380000, 311872, 179900, 225000,
       320000, 219210, 125000, 174000, 143000, 239000, 155000, 192000,
       119000, 144500, 144000, 180500, 192000, 208900, 181000, 214000,
       250000, 240000, 120000, 257500,  62383, 250000, 160000, 275500,
       230000, 392500, 181000, 192000, 245000, 290000, 178000, 275000,
       208900, 281000, 181000, 286000, 165000, 174000, 134000, 250000,
       269790, 285000, 266500, 172500, 183500, 187750, 281000, 237000,
       210000, 120000, 143000, 336000, 192000, 191000, 143000, 501837,
       250000, 170000, 225000, 190000, 231500, 146500, 176000, 170000,
       241500, 190000, 175000, 187750, 230000, 160000, 222500, 176000,
      

In [61]:
err21 = mean_squared_error(y_test, y_pred21)
err21

4071626118.452055

#### 同一モデルのハイパーパラメータ違いを複数組み合わせてみる

In [62]:
clf22 = RandomForestClassifier(n_estimators=50, random_state=1)
clf23 = RandomForestClassifier(n_estimators=80, random_state=2)
clf24 = RandomForestClassifier(n_estimators=100, random_state=3)


In [63]:
eclf22 = VotingClassifier(estimators=[('rf1', clf22), ('rf2', clf23), ('rf3', clf24)], voting='soft')
eclf22 = eclf22.fit(ss_X_train, y_train)

In [64]:
y_pred22 = eclf21.predict(ss_X_test)

In [65]:
err22 = mean_squared_error(y_test, y_pred22)
err22

4071626118.452055

In [51]:
clf25 = RandomForestClassifier(n_estimators=50, random_state=1)
clf26 = RandomForestClassifier(n_estimators=50, random_state=2)
clf27 = RandomForestClassifier(n_estimators=50, random_state=3)


In [52]:
eclf23 = VotingClassifier(estimators=[('rf1', clf25), ('rf2', clf26), ('rf3', clf27)], voting='soft')
eclf23 = eclf23.fit(ss_X_train, y_train)

In [53]:
y_pred23 = eclf23.predict(ss_X_test)

In [54]:
err23 = mean_squared_error(y_test, y_pred23)
err23

4775140491.739726

### <voting='soft', with weight>

ロジスティック回帰はMSEが大きいので除外し、  
②ランダムフォレスト  
③ガウシアンナイーブベース  
④SVM  
の３つで、ウェイトを1：2：1とし、一番単体精度の高いガウシアンナイーブベース の比率を２とする。

In [70]:
eclf3 = VotingClassifier(estimators=[('rf', clf2), ('gnb', clf3), ('svc', clf4)], voting='soft', weights=[1,2,1], flatten_transform=True)

In [71]:
eclf3 = eclf3.fit(ss_X_train, y_train)

In [72]:
y_pred3 = eclf3.predict(ss_X_test)
y_pred3

array([196500, 270000, 120000, 208900, 192000, 244600, 306000, 271900,
       290000, 350000, 176000, 280000, 167500, 232000, 275000, 325300,
       230000, 340000, 230000, 275000, 181000, 279500, 185000, 185000,
       270000, 170000, 161000, 320000, 174500, 132000, 143000, 159000,
       240000, 208900, 177000, 239000, 325000, 311872, 179900, 225000,
       320000, 219210, 134000, 232000, 143000, 239000, 155000, 192000,
       119000, 175000, 144000, 180500, 192000, 208900, 181000, 214000,
       250000, 240000, 120000, 257500, 143000, 230000, 160000, 275500,
       230000, 392500, 181000, 192000, 245000, 290000, 178000, 275000,
       208900, 281000, 181000, 270000, 165000, 232000, 134000, 250000,
       240000, 315000, 266500, 172500, 275000, 176000, 281000, 230000,
       210000, 120000, 143000, 290000, 192000, 191000, 143000, 350000,
       250000, 170000, 225000, 190000, 231500, 140000, 176000, 170000,
       241500, 190000, 179900, 187750, 230000, 185000, 230000, 176000,
      

In [73]:
err3 = mean_squared_error(y_test, y_pred3)
err3

4136419091.9452057

以上の結果から


|       | 単体モデル                | MSE               |
|-------|---------------------------|-------------------|
| Base1 | ①ロジスティック回帰       | 5321997491.726020 |
| Base2 | ②ランダムフォレスト       | 4414022744.479450 |
| Base3 | ③ガウシアンナイーブベース | 4824000759.534240 |
| Base4 | ④SVC                      | 5800495447.205480 |


|           | アンサンブルモデル               |                   |   |
|-----------|----------------------------------|-------------------|---|
| ensemble1 | ①+②+③：head：                    | 5677414021.102730 |   |
| ensemble2 | ①+②+③：soft：                    | 3741653515.712320 | ★ |
| ensemble3 | ②+③+④：soft：                    | 4071626118.452050 | ★ |
| ensemble4 | ②+②+②：soft：                    | 4775140491.739720 | 　 |
| ensemble5 | ②+③+④：soft：waights   = [1,2,1] | 4136419091.945200 | ★ |

(ⅰ)ensemble2で「ロジスティック回帰」+「ランダムフォレスト」+「ガウシアンナイーブベース」の組み合わせでは、  
soft(予測確率のargmax）で3741653515.712320	 
となり、各単体モデルのスコアより高い精度を示している。 

(ⅱ)ensemble3で「ランダムフォレスト」+「ガウシアンナイーブベース」+「SVC」の組み合わせでは、  
soft(予測確率のargmax）で4071626118.452050	 
となり、各単体モデルのスコアより高い精度を示している。

(ⅲ)ensemble5で「ランダムフォレスト」+「ガウシアンナイーブベース」+「SVC」の組み合わせでは、  
比率を1：2：1とすることで、
soft(予測確率のargmax）で4136419091.945200	 
となり、各単体モデルのスコアより高い精度を示している。


# 4.バギング

## 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

### バギングとは
バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。

In [75]:
# 標準化
ss = StandardScaler()
ss.fit(X)
ss_X = ss.transform(X)

In [85]:
from sklearn.tree import DecisionTreeClassifier

# 決定木
model_DT = DecisionTreeClassifier()

X_train_bag, X_test_bag, y_train_bag, y_test_bag = train_test_split(ss_X, y, test_size=0.25, shuffle = True)

# 空行列作成
y_test_bag.reshape(-1,1)
y_pred = np.zeros((y_test_bag.shape[0], 1))

n_iter = 5

for i in range(n_iter):
    # 分割
    X_train_bag, X_test_bag, y_train_bag, y_test_bag = train_test_split(ss_X, y, test_size=0.25, shuffle = True)

    # 学習
    clf = model_DT.fit(X_train_bag, y_train_bag)

    # 推定
    y_pred_WK = model_DT.predict(X_test_bag).reshape(-1,1)

    # predict arrayに追加
    y_pred = np.concatenate([y_pred, y_pred_WK], axis = 1)

print("befor delete y_pred.shape = {}".format(y_pred.shape))
y_pred = np.delete(y_pred, 0, axis = 1)
print("after delete y_pred.shape = {}".format(y_pred.shape))
    
# 推定結果の平均
y_pred_DT = np.mean(y_pred, axis = 1).reshape(-1,1)
print("y_pred_DT.shape = {}".format(y_pred_DT.shape))

# 評価
err_DT = mean_squared_error(y_test, y_pred_DT)
err_DT

befor delete y_pred.shape = (146, 6)
after delete y_pred.shape = (146, 5)
y_pred_DT.shape = (146, 1)


6298982658.290137

In [86]:
from sklearn.tree import DecisionTreeClassifier

## ガウシアンナイーブベース
GNB = GaussianNB()

# 標準化
ss = StandardScaler()
ss.fit(X)
ss_X = ss.transform(X)

X_train_bag, X_test_bag, y_train_bag, y_test_bag = train_test_split(ss_X, y, test_size=0.25, shuffle = True)

# 空行列作成
y_test_bag.reshape(-1,1)
y_pred = np.zeros((y_test_bag.shape[0], 1))
y_true = np.zeros((y_test_bag.shape[0], 1))


n_iter = 5

for i in range(n_iter):
    # 分割
    X_train_bag, X_test_bag, y_train_bag, y_test_bag = train_test_split(ss_X, y, test_size=0.25, shuffle = True)

    # 学習
    GNB.fit(X_train_bag, y_train_bag)

    # 推定
    y_pred_WK = GNB.predict(X_test_bag).reshape(-1,1)
    y_test_bag = y_test_bag.reshape(-1,1)
    
    y_pred = np.concatenate([y_pred, y_pred_WK], axis = 1)
    y_true = np.concatenate([y_true, y_test_bag], axis = 1)

# 正解データ、予測データの最初の列はダミーデータが入っているので削除する。
y_pred = np.delete(y_pred, 0, axis = 1)
y_true = np.delete(y_true, 0, axis = 1)

    
# 正解データ、推定結果の平均
y_pred_DT = np.mean(y_pred, axis = 1).reshape(-1,1)
y_true = np.mean(y_true, axis = 1).reshape(-1,1)
print("y_pred_DT.shape = {}".format(y_pred_DT.shape))

# 評価
err_GNB = mean_squared_error(y_true, y_pred_DT)
err_GNB

y_pred_DT.shape = (146, 1)


959581835.839452

以上をまとめると

| 単一モデル                |                   |
|---------------------------|-------------------|
| モデル                    | MSE               |
| ③ガウシアンナイーブベース | 6298982658.290137 |


| アンサンブル(バギング)    |                   |
|---------------------------|-------------------|
| モデル                    | MSE               |
| ③ガウシアンナイーブベース | 959581835.839452 |

###### 以上の結果となり、ガウシアンナイーブベースの単体のMSEが6298982658.290137であるのに対して、  
バギングを行った後のMSEは、959581835.839452となり、6倍以上の精度向上が見られた。

# 5.スタッキング

## 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

### スタッキングとは

スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。  
まずは 
$K_0$= 3, $M_0$= 2
 程度にします。

#### 《学習時》

（ステージ 0 ）

・学習データを $K_0$個に分割する。  
・分割した内の ($K_0$ − 1) 個をまとめて学習用データ、残り 1個を推定用データとする組み合わせが $K_0$ 個作れる。  
・あるモデルのインスタンスを $K_0$ 個用意し、異なる学習用データを使い学習する。  
・それぞれの学習済みモデルに対して、使っていない残り 1 個の推定用データを入力し、推定値を得る。  
（これをブレンドデータと呼ぶ）  
・さらに、異なるモデルのインスタンスも $K_0$ 個用意し、同様のことを行う。モデルが $M_0$ 個あれば、   
$M_0$ 個のブレンドデータが得られる。  

（ステージ n ）

・ステージ n − 1 のブレンドデータを $M_n−1$ 次元の特徴量を持つ学習用データと考え、 $K_n$ 個に分割する。  
以下同様である。

（ステージ N ）＊最後のステージ

・ステージ N − 1 の $M_{N−1}$ 個のブレンドデータを$M_{N−1}$ 次元の特徴量の入力として、  
1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

#### 《推定時》

（ステージ 0 ）

・テストデータを 
$K_0$ × $M_0$ 個の学習済みモデルに入力し、$K_0$ × $M_0$ 個の推定値を得る。  
これを $K_0$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ n ）

・ステージ n−1 で得たブレンドテストを $K_n$ × $M_n$ 個の学習済みモデルに入力し、$K_n$ × $M_n$ 個の推定値を得る。  
これを $K_n$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ N ）＊最後のステージ

・ステージ N − 1 で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

### 《学習時》

In [137]:
# クロスバリデーション
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso

X_train_list = []
X_test_list = []
y_train_list = []
y_test_list = []
model_ins = []
i = 0
y_brend1 = None
y_brend2 = None
y_brend3 = None
y_brend4 = None

model_ins1 = []
model_ins2 = []


# << stage 0 >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■

# 分割数
K0= 3

# モデル数
M0 = 2

# クロスバリデーション
kf = KFold(n_splits = K0)
kf.get_n_splits(ss_X)

# 変数初期化
y_brend = np.zeros(ss_X.shape[0] * M0).reshape(ss_X.shape[0], M0).reshape(-1, M0)
model_ins_0 = np.empty(K0 * M0).reshape(K0, M0)
model_ins_0 = model_ins_0.astype(np.object)

# インスタンス作成
for ins in range(K0):
    
    model_ins_0[ins,0] = GaussianNB()
    model_ins_0[ins,1] = RandomForestClassifier(n_estimators=50, random_state=1)


# データ分割
for i, (train_index, test_index) in enumerate(kf.split(ss_X)):
    print("i = ", i)
    #print("TRAIN:", train_index, "TEST:", test_index)

        
    X_train, X_test = ss_X[train_index], ss_X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    X_train_list.append(X_train)
    X_test_list.append(X_test)
    y_train_list.append(y_train)
    y_test_list.append(y_test)
    print("X_train.shape = {}".format(X_train.shape))
    print("X_test.shape = {}".format(X_train.shape))
    print("y_train.shape = {}".format(y_train.shape))
    print("y_test.shape = {}".format(y_test.shape))
    
    
    for j in range(M0):
        # 学習
        model_ins_0[i,j].fit(X_train, y_train)

        # 推定
        y_brend[test_index, j] = model_ins_0[i,j].predict(X_test).flatten()
    
print("★★★y_brend = \n{}".format(y_brend))
print("★★★y_brend.shape = {}".format(y_brend.shape))


# << stage n >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
Kn = 3
Mn = 2
n_cnt = 1
kf = KFold(n_splits = Kn)
kf.get_n_splits(X)

# 変数初期化
model_ins_n = np.empty(Kn * Mn).reshape(Kn, Mn)
model_ins_n = model_ins_n.astype(np.object)


# インスタンス作成
for ins in range(K0):
    
    model_ins_n[ins,0] = GaussianNB()
    model_ins_n[ins,1] = RandomForestClassifier(n_estimators=50, random_state=1)

# n回ループ
for n in range(n_cnt):
    for i, (train_index, test_index) in enumerate(kf.split(y_brend)):

        X_train, X_test = y_brend[train_index], y_brend[test_index]
        y_train, y_test = y[train_index], y[test_index]
        X_train_list.append(X_train)
        X_test_list.append(X_test)
        y_train_list.append(y_train)
        y_test_list.append(y_test)


        for j in range(Mn):
            # 学習
            model_ins_n[i,j].fit(X_train, y_train)

            # 推定
            y_brend[test_index, j] = model_ins_n[i,j].predict(X_test).flatten()


# << Last Stage >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
Kl = 3
Ml = 1
n_cnt = 1
kf = KFold(n_splits = Kl)
kf.get_n_splits(X)

# 変数初期化
y_brend_fin = np.zeros(ss_X.shape[0] * Ml).reshape(ss_X.shape[0], Ml).reshape(-1, Ml)
model_ins_l = np.empty(Kl * Ml).reshape(Kl, Ml)
model_ins_l = model_ins_l.astype(np.object)


# インスタンス作成
for ins in range(Kl):
    model_ins_l[ins,0] = Lasso(alpha=0.1)
    #model_ins_l[ins,0] = GaussianNB()
    
for i, (train_index, test_index) in enumerate(kf.split(y_brend)):

    X_train, X_test = y_brend[train_index], y_brend[test_index]
    y_train, y_test = y[train_index], y[test_index]
    X_train_list.append(X_train)
    X_test_list.append(X_test)
    y_train_list.append(y_train)
    y_test_list.append(y_test)

    # 学習
    model_ins_l[i,0].fit(X_train, y_train)

    # 推定
    y_brend_fin[test_index, 0] = model_ins_l[i,0].predict(X_test).flatten()
    
    
print("y_brend_fin.shape = ", y_brend_fin.shape)

i =  0
X_train.shape = (388, 2)
X_test.shape = (388, 2)
y_train.shape = (388,)
y_test.shape = (195,)
i =  1
X_train.shape = (389, 2)
X_test.shape = (389, 2)
y_train.shape = (389,)
y_test.shape = (194,)
i =  2
X_train.shape = (389, 2)
X_test.shape = (389, 2)
y_train.shape = (389,)
y_test.shape = (194,)
★★★y_brend = 
[[157000. 152000.]
 [255000. 186500.]
 [144000. 155000.]
 ...
 [270000. 328900.]
 [250000. 192000.]
 [240000. 259500.]]
★★★y_brend.shape = (583, 2)
y_brend_fin.shape =  (583, 1)


### 《推定時》

In [138]:
# << stage 0 >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
predict_model_0 = np.zeros(ss_X.shape[0] * M0).reshape(ss_X.shape[0], -1)
for m in range(M0):
    predict_instance_0 = np.zeros(ss_X.shape[0] * K0).reshape(ss_X.shape[0], -1)
    
    for k in range(K0):
        # 推定
        predict_instance_0[:,k] = model_ins_0[k,m].predict(ss_X)
        
    predict_model_0[:,m] = np.mean(predict_instance_0, axis = 1).flatten()

print("predict_model_0 = \n", predict_model_0)

# << stage n >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
# n回ループ
for n in range(n_cnt):
    predict_model_n = np.zeros(ss_X.shape[0] * Mn).reshape(ss_X.shape[0], -1)
    predict_model_n = predict_model_0
    for m in range(M0):
        predict_instance_n = np.zeros(ss_X.shape[0] * Kn).reshape(ss_X.shape[0] , -1)

        for k in range(K0):
            predict_instance_n[:,k] = model_ins_n[k,m].predict(predict_model_n)
            
        predict_model_n[:,m] = np.mean(predict_instance_n, axis = 1).flatten()
        
print("predict_model_n = \n", predict_model_n)

# << Last Stage >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
# 推定
y_predict_fin = model_ins_l[i,0].predict(predict_model_n).flatten()
y_predict_fin.reshape(ss_X.shape[0], -1)

print("y_predict_fin.shape", y_predict_fin.shape)
print("y_predict_fin = \n", y_predict_fin)


predict_model_0 = 
 [[173333.33333333 171666.66666667]
 [236166.66666667 211166.66666667]
 [142666.66666667 145000.        ]
 ...
 [222333.33333333 226300.        ]
 [265000.         204000.        ]
 [257666.66666667 264166.66666667]]
predict_model_n = 
 [[189500.         205366.66666667]
 [233833.33333333 232833.33333333]
 [152000.         120666.66666667]
 ...
 [188500.         273166.66666667]
 [214333.33333333 254333.33333333]
 [278333.33333333 257626.        ]]
y_predict_fin.shape (583,)
y_predict_fin = 
 [213012.95846678 250800.92383616 163177.93054025 272897.02615678
 275136.79515987 163155.52300313 147916.93320262 285073.9768373
 272105.70463502 278861.27812632 145137.48173903 262732.16407632
 145827.21020453 304844.21236439 278861.27812632 234346.38427826
 209321.6032782  306390.30768016 332817.62708949 176100.7921859
 217807.53569107 192767.73737062 272897.02615678 290788.33917638
 152753.30584235 327990.18416993 167691.46525483 199893.42922787
 365642.80948309 282095.514075

In [139]:
# 評価
MSE = mean_squared_error(y, y_predict_fin)
MSE

3232191779.8704348

### 以上をクラス化する。

In [168]:
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso

class ScratchStacking():

    def __init__(self, verbose=False):
        # ハイパーパラメータを属性として記録
        self.K0 = 3
        self.M0 = 2
        self.Kn = 3
        self.Mn = 2
        self.n_cnt = 10
        self.Kl = 3
        self.Ml = 1
        self.verbose = verbose
        
    def fit(self, X, y):
        X_train_list = []
        X_test_list = []
        y_train_list = []
        y_test_list = []
        model_ins = []

        model_ins1 = []
        model_ins2 = []

        # << stage 0 >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
        # クロスバリデーション
        kf = KFold(n_splits = self.K0)
        kf.get_n_splits(ss_X)

        # 変数初期化
        y_brend = np.zeros(X.shape[0] * self.M0).reshape(X.shape[0], self.M0).reshape(-1, self.M0)
        model_ins_0 = np.empty(self.K0 * self.M0).reshape(self.K0, self.M0)
        model_ins_0 = model_ins_0.astype(np.object)

        # インスタンス作成
        for ins in range(self.K0):
            model_ins_0[ins,0] = GaussianNB()
            model_ins_0[ins,1] = RandomForestClassifier(n_estimators=50, random_state=1)


        # データ分割
        for i, (train_index, test_index) in enumerate(kf.split(X)):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]


            for j in range(self.M0):
                # 学習
                model_ins_0[i,j].fit(X_train, y_train)

                # 推定
                y_brend[test_index, j] = model_ins_0[i,j].predict(X_test).flatten()

        # << stage n >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
        self.n_cnt = 1
        kf = KFold(n_splits = self.Kn)
        kf.get_n_splits(X)

        # 変数初期化
        model_ins_n = np.empty(self.Kn * self.Mn).reshape(self.Kn, self.Mn)
        model_ins_n = model_ins_n.astype(np.object)

        # インスタンス作成
        for ins in range(self.K0):
            model_ins_n[ins,0] = GaussianNB()
            model_ins_n[ins,1] = RandomForestClassifier(n_estimators=50, random_state=1)

        # n回ループ
        for n in range(self.n_cnt):
            for i, (train_index, test_index) in enumerate(kf.split(y_brend)):
                X_train, X_test = y_brend[train_index], y_brend[test_index]
                y_train, y_test = y[train_index], y[test_index]

                for j in range(self.Mn):
                    # 学習
                    model_ins_n[i,j].fit(X_train, y_train)

                    # 推定
                    y_brend[test_index, j] = model_ins_n[i,j].predict(X_test).flatten()

        # << Last Stage >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
        kf = KFold(n_splits = self.Kl)
        kf.get_n_splits(X)

        # 変数初期化
        y_brend_fin = np.zeros(X.shape[0] * self.Ml).reshape(X.shape[0], self.Ml).reshape(-1, self.Ml)
        model_ins_l = np.empty(self.Kl * self.Ml).reshape(self.Kl, self.Ml)
        model_ins_l = model_ins_l.astype(np.object)

        # インスタンス作成
        for ins in range(self.Kl):
            model_ins_l[ins,0] = Lasso(alpha=0.1)

        for i, (train_index, test_index) in enumerate(kf.split(y_brend)):

            X_train, X_test = y_brend[train_index], y_brend[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # 学習
            model_ins_l[i,0].fit(X_train, y_train)

            # 推定
            y_brend_fin[test_index, 0] = model_ins_l[i,0].predict(X_test).flatten()

        if self.verbose:
            print("y_brend_fin.shape = ", y_brend_fin.shape)
        
        
    def predict(self, X):
        # << stage 0 >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
        predict_model_0 = np.zeros(X.shape[0] * self.M0).reshape(X.shape[0], -1)
        for m in range(self.M0):
            predict_instance_0 = np.zeros(X.shape[0] * self.K0).reshape(X.shape[0], -1)

            for k in range(self.K0):
                # 推定
                predict_instance_0[:,k] = model_ins_0[k,m].predict(X)

            predict_model_0[:,m] = np.mean(predict_instance_0, axis = 1).flatten()

        # << stage n >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
        # n回ループ
        for n in range(self.n_cnt):
            predict_model_n = np.zeros(X.shape[0] * self.Mn).reshape(X.shape[0], -1)
            predict_model_n = predict_model_0
            for m in range(self.M0):
                predict_instance_n = np.zeros(X.shape[0] * self.Kn).reshape(X.shape[0] , -1)

                for k in range(self.K0):
                    predict_instance_n[:,k] = model_ins_n[k,m].predict(predict_model_n)

                predict_model_n[:,m] = np.mean(predict_instance_n, axis = 1).flatten()


        # << Last Stage >> ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
        # 推定
        y_predict_fin = model_ins_l[i,0].predict(predict_model_n).flatten()
        y_predict_fin.reshape(X.shape[0], -1)
        
        return y_predict_fin


In [173]:
ScratchStack = ScratchStacking()
ScratchStack.fit(ss_X, y)

In [174]:
y_pred = ScratchStack.predict(ss_X)
#y_pred

In [175]:
# 評価
MSE = mean_squared_error(y, y_pred)
MSE

3232191779.8704348

結果は上記のようになり、これまでのブレンディング/バギングと比べてスタッキングで最高の精度3232191779.8704348が出た。